In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [4]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-12

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':1,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [5]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [6]:
points = [x0]
 
for _ in range(4):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [ ]:
#run_alpha_policies = ["standard","line_search","icml"]
run_alpha_policies = ['new_lloo','lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
5.232711198171899
iter = 1, stepsize = 0.3917318246754077, criterion = 0.01, upper_bound=0.05635009446772796, lower_bound=-5.176361103704171, real_Gap=5.232711198171899
iter = 1000, stepsize = 0.0011439559681520565, criterion = 0.0009248454983790988, upper_bound=-4.571409662600624, lower_bound=-4.571699455146176, real_Gap=0.0002897925455513928
iter = 2000, stepsize = 0.0007198143544010637, criterion = 0.0004535517143560335, upper_bound=-4.571412618402145, lower_bound=-4.571519133437572, real_Gap=0.00010651503542735696
iter = 3000, stepsize = 0.0004789259169793097, criterion = 0.0003023332497081548, upper_bound=-4.571413139630295, lower_bound=-4.571460408543107, real_Gap=4.7268912812370445e-05
iter = 4000, stepsize = 0.0005382770017464023, criterion = 0.0002257643857725455, upper_bound=-4.57141332361432, lower_bound=-4.571432137763861, real_Gap=1.8814149541412917e-05
iter = 5000, stepsize = 0.0002266682508409376, criterion = 0.00018066708752570985, u

In [ ]:
run_alpha_policies = ["standard","line_search","icml"]

for policy in run_alpha_policies:
    policy_results = []
    for i in range(len(points)):
        x0 = points[i]
        x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
        policy_results.append(
            {
        'Q_hist': Q_hist,
        'time_hist': time_hist,}
        )
    with open('./results/portfolio/' + problem_name + '_' + policy + '.pckl', 'wb') as f:
        pickle.dump(policy_results, f)
#    results[problem_name][policy] = policy_results

## Run SCOPT

In [ ]:
scopt_results = []
for i in range(len(points)):
    x0 = points[i]
    x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
            grad_x,
            hess_mult_x,
            hess_mult_vec_x,
            Mf,
            nu,
            prox_func,
            x0,  
            sc_params,                                              
            eps=terminate_tol,                                              
            print_every=1)
        
    scopt_results.append({
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    })